Tipo da variável: categórica ou numérica


*GERAL*

*Visões geral*:

* Quantidade de inscritos por ano - OK
* Percentual de ausentes nos dois dias - OK
* Inscritos por tipo de língua - OK
* Inscritos por escola (imagino que é por tipo de escola) - OK
* Inscritos por gênero - OK
* Inscritos por depedência adminstrativa da escola (Inclui) - OK
* Composição da quantidade de inscritos no por ano
    * Proporção de gênero - OK
    * Proporção de escolas - OK
    * Proporção dependência administrativa da escola - OK
    * ...
* Status da Redação - NÃO ENTENDI
    
*Visão Série Temporal*

* Quantidade de inscritos na série histórica (2010-2021) - OK
* Sexo - OK
* Cor/Raça - OK
* Estado Civil - OK
* Conclusão Ensino Médio - NÃO ENTENDI
* Tipo Escola - OK
* Dependência administrativa - OK
* Média nota no tempo por variável
    * Ano (Inclui) - OK
    * Tipo de lingua (inclui) - OK
    * Sexo - OK
    * Tipo escola - OK
    * Dependência administrativa - OK
    * Estado Civil (inclui) - OK
    * Região geográfica - OK
    * UF - OK
    * Município - OK
    * Cor/raça - OK
    * Renda - Ver com Lucas
    * Escolaridade do pai/mãe - Ver como fazer com Lucas pois são duas variáveis.
    * Tamanho da família - 

*Composição Geral*

Proporção e valor absoluto
* Sexo e localidade (UF/Região geográfica)
* Cor/Raça
* Tipo Escola
* Dependência administrativa 
    
*Visão Média Geral*

* Média global
* Média por gênero
* Média por etnia
* Média por tipo de escola
* Média por tempo de conclusão
* Média por faixa etária
* Média/Nota Máxima por regiões

*Visão abstenção/falta*

* Falta por gênero
* Falta por regiões geográficas
* Falta por UF
* Falta por município
* Falta por faixa etária
* Falta por disciplina
* Falta por etnia
* Falta por tipo de escola


*ESTATÍSTICA*
* Univariada
    * Filtros : Prova / Ano / Variável
    * Boxplot pela variável
    * Gráfico das notas densidade por cada grupo da variável
    * Gráfico de Barras
        * Quebrar por sexo
    * Situação de conclusão
    * Escolaridade do pai/mãe
    * Tamanho da família
    * Acesso à internet
    * Computador
* Análise multivariada

*RANKING*

    
* Quantidade de inscritos na Bahia nos últimos anos
* Composição da quantidade de inscritos na Bahia
* Mapa com a quantidade de inscritos (2019-2020) por UF
* Mapa com quantidade de inscritos por município (2019-2020)

* Ranking de Média nas provas por UF pré-pandemia (2019) e durante a pandemia (2020)
* Ranking de Média nas provas por município pré-pandemia e durante a pandemia
* 

* Visão UF
    * Quantidade de Inscritos
    * Falta
    * Proporções
* Visão Município
* Visão Escola

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import os

In [2]:
os.chdir("C:\\Users\\brunobezerra\\Downloads\\Livros e papers\\Microdados ENEM\\Microdados")


In [3]:
pwd

'C:\\Users\\brunobezerra\\Downloads\\Livros e papers\\Microdados ENEM\\Microdados'

#### Funções.

In [4]:
def create_columns_region(uf):
    norte        = ['AM', 'RR', 'AP', 'PA', 'TO', 'RO', 'AC']
    nordeste     = ['MA', 'PI', 'CE', 'RN', 'PE', 'PB', 'SE', 'AL', 'BA']
    centro_oeste = ['MT', 'MS', 'GO']
    sudeste      = ['SP', 'RJ', 'ES', 'MG']
    sul          = ['PR', 'RS', 'SC']
    df           = ['DF']
    
    if uf in norte :
        return 'Norte'
    elif uf in nordeste :
        return 'Nordeste'
    elif uf in centro_oeste : 
        return 'Centro-Oeste'
    elif uf in sudeste : 
        return 'Sudeste'
    elif uf in sul : 
        return 'Sul'
    elif uf in df :
        return 'DF'
    else:
        return 'missing'
    
    
    
def build_statistics_metrics(ano_enem : int, 
                             lista_provas : list, 
                             df_enem : pd.DataFrame, 
                             cols_to_group : list, 
                             list_with_metrics = None):
    if list_with_metrics:
        list_metrics = list_with_metrics
    else:
        list_metrics = { 'MIN_': np.min, 'AVG_' :np.mean, 'MED_' : np.median, 'MAX_' : np.max, 'STD_' : np.std}
    
    def get_statistics(ano_enem, lista_provas, metric, df_enem, cols_to_group, metric_label):

        agg_result = (df_enem
         .query(f'NU_ANO == {ano_enem}')
         .groupby(cols_to_group, as_index = False)
         .agg({prova:metric for prova in lista_provas})
         .rename(columns = {prova : metric_label+'_'.join(prova.split('_')[1:]) for prova in lista_provas})
        )

        return agg_result
    
    result = [
        get_statistics(ano_enem, lista_provas, list_metrics[metric], df_enem, cols_to_group, metric).set_index(cols_to_group) 
      for metric in list_metrics
     ]

    result_df = (pd.concat(result, axis=1)
         .reset_index()
        )
    

    return result_df


def build_distribution_freq(df_enem, cols_to_group, cat_columns):
    
    def get_distribution_freq(df_enem, cols_to_group, cat_column):
        agg_result = (pd.get_dummies(df_enem[cols_to_group+[cat_column]],
               columns = [cat_column])
                 .groupby(cols_to_group).sum()
                )
        return agg_result
    
    result    = [get_distribution_freq(df_enem, cols_to_group, cat_column) for cat_column in cat_columns]
    result_df = (pd.concat(result, axis = 1))
    return result_df

def get_count(ano_enem, df_enem, cols_to_group, column, column_name = None):
    agg_result = (df_enem
         .query(f'NU_ANO == {ano_enem}')
         .groupby(cols_to_group)
         .agg({column:'count'})
         .rename(columns = {column : column_name} )
        )
    return agg_result

#### Coleta de dados

In [9]:
#Teste
ano_enem = 2018
df_enem  = pd.read_csv(f'MICRODADOS_ENEM_{ano_enem}.csv', encoding = 'latin1', sep = ';')

: 

In [5]:
df_enem  = pd.read_csv(f'MICRODADOS_ENEM_2022.csv', encoding = 'latin1', sep = ';')

In [5]:
df=pd.read_csv('MICRODADOS_ENEM_2022.csv', sep=';', encoding='latin-1')

In [ ]:
pd.set_option('display.max_columns', None)  # Configura para exibir todas as colunas
df.head(5)

#### Baixar dataframe

In [ ]:
df.to_excel('D:\Repositório GIT\Projetos\IBGE\Datasets\Educação\Microdado_Teste1.xlsx', index=False)